In [40]:
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold,  cross_val_score
from sklearn.metrics import make_scorer, fbeta_score

In [41]:
# Lade Datensatz
train_data_loaded = pd.read_csv('../../data/train_data_2024-08-01.csv')
X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']


In [42]:
# StratifiedKFold initialisieren
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Variablen für Ergebnisse
fbetas_rf = []
fbetas_random = []
fbetas_mehrheit = []

In [43]:
# Liste von beta-Werten
beta_values = [0.7, 0.9, 1, 2, 3, 5, 10, 15, 20, 30, 50]

# Variablen für Ergebnisse
results = {}
results_rf = {}
results_random = {}
results_mehrheit = {}
results_rf_recall = {}
results_random_recall = {}
results_majority_recall = {}
results_rf_precision = {}
results_random_precision = {}
results_majority_precision = {}


In [44]:
from sklearn.metrics import recall_score, precision_score

for beta in beta_values:
    fbeta_scorer = make_scorer(fbeta_score, beta=beta)
    model_logreg = LogisticRegression(C=1, max_iter=1700, penalty='l2', solver='liblinear', tol=0.0001, random_state=42, class_weight={0: 1, 1: 9})
    logreg_scores = cross_val_score(model_logreg, X, y, cv=skf, scoring=fbeta_scorer)
    results[beta] = logreg_scores.mean()

   # print(f'LogReg F betas für beta = {beta}: {logreg_scores}')
   # print(f'LogReg F-beta Score (mean) für beta = {beta}: {logreg_scores.mean()}')

    # Variablen für Ergebnisse
    fbetas_rf = []
    fbetas_random = []
    fbetas_mehrheit = []
    recall_rf = []
    recall_random = []
    recall_majority = []
    precision_scores_rf = []
    precision_scores_random = []
    precision_scores_majority = []
    
    counter = 1
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Drucke die ersten 10 values von y_train
        print(f"Die ersten 20 Werte von y_train für beta = {beta}:               ", y_train.values[:20])
        # Drucke die ersten 10 values von y_test
        print(f"Die ersten 20 Werte von y_test für beta = {beta}:                ", y_test.values[:20])
        
        # Random Forest Modell trainieren
        model_logreg.fit(X_train, y_train)
        y_pred_rf = model_logreg.predict(X_test)
        print(f"Ersten 20 Vorhersagen mit dem Modell für beta = {beta}:          ", y_pred_rf[:20])
        recall_rf.append(recall_score(y_test, y_pred_rf))      
        precision_scores_rf.append(precision_score(y_test, y_pred_rf))
        
        # Zufällige Vorhersagen basierend auf den Klassenwahrscheinlichkeiten erstellen
        class_counts = Counter(y_test)
        total_samples = len(y_test)
        class_probabilities = {cls: count / total_samples for cls, count in class_counts.items()}
        
        np.random.seed(42)
        y_pred_random_weighted = np.random.choice(
            list(class_probabilities.keys()),
            size=y_test.shape,
            p=list(class_probabilities.values())
        )
        
        recall_random.append(recall_score(y_test, y_pred_random_weighted)) 
        precision_scores_random.append(precision_score(y_test, y_pred_random_weighted))
        
        # Drucke die ersten 10 Werte
        print(f"Ersten 20 Vorhersagen mit dem _random_weighted für beta = {beta}:", y_pred_random_weighted[:20])
        
        # Leistung der gewichteten zufälligen Vorhersagen bewerten
        fbeta_random_weighted = fbeta_score(y_test, y_pred_random_weighted, beta=beta )
      
        fbetas_random.append(fbeta_random_weighted)
         
        # Immer Mehrheitsklasse vorhersagen
        y_pred_majority = np.full(y_test.shape, y_train.mode().iloc[0])
        
        recall_majority.append(recall_score(y_test, y_pred_majority))
        precision_scores_majority.append(precision_score(y_test, y_pred_majority))
        
        # Drucke die ersten 10 Werte
        print(f"Ersten 20 Vorhersagen mit dem y_pred_majority für beta = {beta}: ", y_pred_majority[:20])
        
        # Leistung der Vorhersagen der Mehrheitsklasse bewerten
        fbeta_mehrheit = fbeta_score(y_test, y_pred_majority, beta=beta)
        
       # print(f"Fold Nummer {counter}: fbeta_random_weighted - Score ist {fbeta_random_weighted}")
       # print(f"Fold Nummer {counter}: fbeta_mehrheit - Score ist {fbeta_mehrheit}")
        fbetas_mehrheit.append(fbeta_mehrheit)
        
        
        counter += 1
        
    results_rf[beta] = np.mean(logreg_scores)
    results_random[beta] = np.mean(fbetas_random)
    results_mehrheit[beta] =  np.mean(fbetas_mehrheit)
    results_rf_recall[beta] = np.mean(recall_rf) 
    results_random_recall[beta] = np.mean(recall_random)
    results_majority_recall[beta] = np.mean(recall_majority)
    results_rf_precision[beta] = np.mean(precision_scores_rf)
    results_random_precision[beta] = np.mean(precision_scores_random)
    results_majority_precision[beta] = np.mean(precision_scores_majority)
    
       

Die ersten 20 Werte von y_train für beta = 0.7:                [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
Die ersten 20 Werte von y_test für beta = 0.7:                 [0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
Ersten 20 Vorhersagen mit dem Modell für beta = 0.7:           [1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0]
Ersten 20 Vorhersagen mit dem _random_weighted für beta = 0.7: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
Ersten 20 Vorhersagen mit dem y_pred_majority für beta = 0.7:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Die ersten 20 Werte von y_train für beta = 0.7:                [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0]
Die ersten 20 Werte von y_test für beta = 0.7:                 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0]


/home/lisa/Code/Python/data_science1/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Ersten 20 Vorhersagen mit dem Modell für beta = 0.7:           [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Ersten 20 Vorhersagen mit dem _random_weighted für beta = 0.7: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
Ersten 20 Vorhersagen mit dem y_pred_majority für beta = 0.7:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Die ersten 20 Werte von y_train für beta = 0.7:                [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0]
Die ersten 20 Werte von y_test für beta = 0.7:                 [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0]


/home/lisa/Code/Python/data_science1/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Ersten 20 Vorhersagen mit dem Modell für beta = 0.7:           [1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Ersten 20 Vorhersagen mit dem _random_weighted für beta = 0.7: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
Ersten 20 Vorhersagen mit dem y_pred_majority für beta = 0.7:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Die ersten 20 Werte von y_train für beta = 0.7:                [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
Die ersten 20 Werte von y_test für beta = 0.7:                 [0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0]


...

/home/lisa/Code/Python/data_science1/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Ersten 20 Vorhersagen mit dem Modell für beta = 50:           [1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1]
Ersten 20 Vorhersagen mit dem _random_weighted für beta = 50: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
Ersten 20 Vorhersagen mit dem y_pred_majority für beta = 50:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Die ersten 20 Werte von y_train für beta = 50:                [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Die ersten 20 Werte von y_test für beta = 50:                 [0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0]


/home/lisa/Code/Python/data_science1/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Ersten 20 Vorhersagen mit dem Modell für beta = 50:           [1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1]
Ersten 20 Vorhersagen mit dem _random_weighted für beta = 50: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
Ersten 20 Vorhersagen mit dem y_pred_majority für beta = 50:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


/home/lisa/Code/Python/data_science1/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [45]:

# Ergebnisse ausgeben
for beta in results.keys():
    print(f'Beta: {beta}')
    print(f'F-beta Score (mean) logReg: {results[beta]}')
    print(f'Recall Score logReg: {results_rf_recall[beta]}')
    print(f'Precision Score logReg: {results_rf_precision[beta]}')
    print()
 #   print(f'F-beta Score (mean) logReg händischer StratifiedFold: {results_rf[beta]}')
    print(f'F-beta Score (mean) Weighted Random Prediction: {results_random[beta]}')
    print(f'Recall Score Weighted Random Prediction: {results_random_recall[beta]}')
    print(f'Precision Score Weighted Random Prediction: {results_random_precision[beta]}')
    print()
    print(f'F-beta Score (mean) Mehrheit Prediction: {results_mehrheit[beta]}')
    print(f'Recall Score Mehrheit Prediction: {results_majority_recall[beta]}')
    print(f'Precision Score Mehrheit Prediction: {results_majority_precision[beta]}')
    print()
    print()


Beta: 0.7
F-beta Score (mean) logReg: 0.2339360085670165
Recall Score logReg: 0.8785124806144303
Precision Score logReg: 0.1720731704012424

F-beta Score (mean) Weighted Random Prediction: 0.15356285319118382
Recall Score Weighted Random Prediction: 0.1508846149788473
Precision Score Weighted Random Prediction: 0.1549102029469002

F-beta Score (mean) Mehrheit Prediction: 0.0
Recall Score Mehrheit Prediction: 0.0
Precision Score Mehrheit Prediction: 0.0


Beta: 0.9
F-beta Score (mean) logReg: 0.26880414850039197
Recall Score logReg: 0.8785124806144303
Precision Score logReg: 0.1720731704012424

F-beta Score (mean) Weighted Random Prediction: 0.15308245532799905
Recall Score Weighted Random Prediction: 0.1508846149788473
Precision Score Weighted Random Prediction: 0.1549102029469002

F-beta Score (mean) Mehrheit Prediction: 0.0
Recall Score Mehrheit Prediction: 0.0
Precision Score Mehrheit Prediction: 0.0


Beta: 1
F-beta Score (mean) logReg: 0.28777772921160116
Recall Score logReg: 0.87

In [46]:

# Klassifikationsberichte für den letzten Fold ausgeben
print("Random Forest Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_rf))

print("Weighted Random Prediction Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_random_weighted))

Random Forest Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.91      0.23      0.37      6787
           1       0.17      0.87      0.29      1231

    accuracy                           0.33      8018
   macro avg       0.54      0.55      0.33      8018
weighted avg       0.80      0.33      0.36      8018

Weighted Random Prediction Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      6787
           1       0.16      0.15      0.15      1231

    accuracy                           0.74      8018
   macro avg       0.50      0.50      0.50      8018
weighted avg       0.74      0.74      0.74      8018

